# Programming Objective

1. Use the movies data made famous by ggplot2 `dbfs:/databricks-datasets/Rdatasets/data-001/csv/ggplot2/movies.csv`.  
2. Filter to movies that occurred between January 1, 1975, and December 31, 2000, with a reported budget.
3. Calculate the z-score for each movie by year (x - mean(x) / sd(x)), and round the z-score to 4 decimal points.
4. Rank the movies with the most expensive being a rank of 1 within each year. Arrange the table by rank and display this table with the following columns shown - `["title", "year", "length", "budget", "zscore", "rank"]`.
5. Create a smaller table with the top three most expensive movies by year. Sort the table by z-score and display the entire table with the following columns shown - `["title", "year", "length", "budget", "zscore", "rank"]`.

In [0]:
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from plotnine import *
import pyspark
from pyspark.sql import SparkSession

**1. Use the movies data made famous by ggplot2 `dbfs:/databricks-datasets/Rdatasets/data-001/csv/ggplot2/movies.csv`.**

In [0]:
dat = spark.read.csv("dbfs:/databricks-datasets/Rdatasets/data-001/csv/ggplot2/movies.csv", header = True, nullValue='NA')
display(dat)

**2. Filter to movies that occurred between January 1, 1975, and December 31, 2000, with a reported budget.**

In [0]:
#Dropping all null values in column "budget". This is necessary in order to filter all movies with at least a reported budget.
dropped_nullV_data = dat.na.drop(subset=['budget'])

#Filter movies by year between 1975 and 2000.
movies_data = dropped_nullV_data.filter((dropped_nullV_data['year'] >= 1975) & (dropped_nullV_data['year'] <= 2000))

display(movies_data)

**3. Calculate the z-score for each movie by year (x - mean(x) / sd(x)), and round the z-score to 4 decimal points.**

In [0]:
#This function will return the z-score of each movie based on the budget and partitioning by year.
def get_z_score(budget, window_partition):
    avg = F.avg(budget).over(window_partition)
    sd = F.stddev(budget).over(window_partition)
    return (budget - avg) / sd

window_partition = Window().partitionBy("year")
z_scored_data = movies_data.withColumn("zscore", F.round(get_z_score(movies_data.budget, window_partition), 4))
z_scored_movies_data = z_scored_data.select("title", "year", "length", "budget", "zscore")

z_scored_movies_data.printSchema()

z_scored_movies_data = z_scored_movies_data.withColumn("budget", F.col("budget").cast("int"))

display(z_scored_movies_data)

**4. Rank the movies with the most expensive being a rank of 1 within each year. Arrange the table by rank and display this table with the following columns shown - `["title", "year", "length", "budget", "zscore", "rank"]`.**

In [0]:
partition_by_most_expensive = Window.partitionBy("year").orderBy(F.col("budget").desc())
ranked_most_expensive_movies = z_scored_movies_data.withColumn("rank", F.rank().over(partition_by_most_expensive))

display(ranked_most_expensive_movies)

**5. Create a smaller table with the top three most expensive movies by year. Sort the table by z-score and display the entire table with the following columns shown - `["title", "year", "length", "budget", "zscore", "rank"]`.**

In [0]:
top_3_most_expensive_movies_by_year = ranked_most_expensive_movies.filter(F.col("rank") <= 3).drop("row")
display(top_3_most_expensive_movies_by_year)

In [0]:
#Sort the table by z-score. Sorting from largest to smallest
sorted_data_by_ZScore = top_3_most_expensive_movies_by_year.sort(F.col("zscore").desc())
display(sorted_data_by_ZScore)